In [2]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import gc
import os

In [3]:
# Load test_sup and find index
/home/kai/data/kaggle/homecredit
load_path = '/home/kai/data/kaggle/talkingdata/downsampling/'                     
test_supplement = pd.read_feather(load_path + 'test_cleaned.ftr')
print('sup read done')
test_index = np.load('/home/kai/data/kaggle/talkingdata/data/supplement2testInexFinal.npy')
print('slicing without correction done')
test = test_supplement.iloc[test_index]
print(test.shape)

# # Test_id correction
# test_clickid = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv', usecols=['click_id'])
# print('test_clickid done')
# test['click_id'] = test_clickid['click_id']

sup read done
slicing without correction done
(18790469, 106)


In [4]:
print(test_supplement.shape)

(57537505, 106)


In [5]:
load_path = '/home/kai/data/kaggle/talkingdata/downsampling/'
file_format = '{}.ftr'
train_dict = {}
train_indexlist = ['0', '1', '2', '3', '4']
    
for file in train_indexlist:
    print(file)
    train_dict[file] = pd.read_feather(load_path + file_format.format(file))
    
print('train(downsampled) done')
val = pd.read_feather(load_path + 'val.ftr')
print('val done')

0
1
2
3
4
train(downsampled) done
val done


In [6]:
print((train_dict['0'].shape))

(794952, 106)


In [12]:
target = 'is_attributed'

combine = 0

params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 4000,
        'learning_rate': 0.1,
        'num_leaves': 61,
        'num_threads': 16, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 390,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.7, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':60,
        'bagging_fraction': 0.7, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        'verbose': 0,
        'scale_pos_weight': 1,
        'metric' : [ 'auc']
    }

# params = {
#     'boosting_type': 'gbdt',
#     'colsample_bytree':0.39,
#     'learning_rate': 0.05,
#     'max_bin': 305,
#     'metric': ['auc'], ##
#     'min_child_samples': 20,
#     'min_child_weight': 5,
#     'min_split_gain': 0,
#     'num_threads': 16,
#     'num_leaves': 74,
#     'objective':'binary',
#     'reg_alpha':0,
#     'reg_lambda':0.5,
#     'scale_pos_weight': 1,
#     'subsample':1.0,
#     'subsample_for_bin':200000,
#     'subsample_freq':1,
#     'verbose':0,
#     'num_boost_round': 2000,
#     'early_stopping_round': 100,
#     'verbose_eval': 10
    
#     }
# categorical_col = ['os', 'hour', 'device', 'channel', 'app', 'com1_ip']
categorical_col = [ 'app', 'device', 'os', 'channel', 'hour']
feature_cols = list(test.columns.values)
print(len(feature_cols))
remove_list = ['index', 'ip', 'click_id', 'click_time', 'is_test', 'minute10', 'hourminute10', 'dayhourminute', 'dayhourminute10',\
              'machine', 'click_timestamp', target]
feature_cols = [x for x in feature_cols if x not in remove_list]
print(feature_cols)
print(len(feature_cols))

106
['app', 'device', 'os', 'channel', 'day', 'hour', 'minute', 'hourminute', 'nunique_day_ip_app', 'woe_ip_channel_nextClickLeakDayFlt', 'woe_app_device_nextClickLeakDayFlt', 'variance_ip_device_hour', 'woe_ip_app', 'count_ip_app_os_channel_day', 'woe_ip_nextClickLeakDayFlt', 'com1_ip', 'woe_app_channel', 'woe_ip_app_device_os', 'nunique_day_ip_machine', 'rang_count_ip_device_os_dayhourminute', 'count_ratio_channel_app', 'woe_os_channel_nextClickLeakDayFlt', 'woe_app_device_channel', 'woe_os_nextClickLeakDayFlt', 'woe_channel', 'count_ip', 'woe_app_device', 'woe_app_os', 'woe_app_device_os', 'woe_ip_device_os', 'woe_app_device_os_channel', 'rang_count_ip_day', 'nunique_machine_channel', 'woe_ip', 'count_ip_app_device_day', 'cumcount_ip_day', 'rang_count_ip_hour', 'cumcount_ip_app_device_os_day_hour', 'rang_count_ip_channel_dayhourminute10', 'nunique_day_ip_os', 'woe_ip_device_channel', 'woe_ip_app_device', 'count_app_os_channel_day_hour', 'woe_device_channel_nextClickLeakDayFlt', 'cum

In [13]:
def train_lightgbm(x_train, x_val, feature_cols, categorical_feature, params, best_round = None, target='is_attributed'):
    param = params.copy()
    y_train = x_train[target].values
    y_val = x_val[target].values
    
    lgb_train = lgb.Dataset(x_train[feature_cols], y_train, categorical_feature = categorical_feature)
    lgb_val = lgb.Dataset(x_val[feature_cols], y_val, categorical_feature = categorical_feature)
#     if best_round is not None:
#         param['num_rounds'] = best_round
#         del param['early_stopping_round']
    print('start training')
    model = lgb.train(param, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)
    return model


# Train model and get prediction

In [14]:
pred = {}
for i in train_indexlist:
    print(i)
    model = train_lightgbm(train_dict[i], val, feature_cols, categorical_col, params)
    # best_round = model.best_iteration
    print(i + 'th' +'model trained' )
    # pred = model.predict(test)
    pred[i] = model.predict(test[feature_cols])
print('prediction done')
# np.save(load_path+file_name, pred)




0
start training


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 60 rounds.
[10]	valid_0's auc: 0.981794
[20]	valid_0's auc: 0.983339
[30]	valid_0's auc: 0.984358
[40]	valid_0's auc: 0.985049
[50]	valid_0's auc: 0.985391
[60]	valid_0's auc: 0.985628
[70]	valid_0's auc: 0.985862
[80]	valid_0's auc: 0.985982
[90]	valid_0's auc: 0.98608
[100]	valid_0's auc: 0.986106
[110]	valid_0's auc: 0.986112
[120]	valid_0's auc: 0.986115
[130]	valid_0's auc: 0.986099
[140]	valid_0's auc: 0.986094
[150]	valid_0's auc: 0.986123
[160]	valid_0's auc: 0.986098
[170]	valid_0's auc: 0.986087
Early stopping, best iteration is:
[117]	valid_0's auc: 0.986146
0thmodel trained
1
start training
Training until validation scores don't improve for 60 rounds.
[10]	valid_0's auc: 0.98193
[20]	valid_0's auc: 0.983328
[30]	valid_0's auc: 0.984365
[40]	valid_0's auc: 0.985016
[50]	valid_0's auc: 0.985368
[60]	valid_0's auc: 0.985588
[70]	valid_0's auc: 0.98576
[80]	valid_0's auc: 0.985841
[90]	valid_0's auc: 0.985925
[100]	valid_0's au

In [15]:
pred_blend = (pred['1'] + pred['0'] + pred['2'] + pred['3'] + pred['4'])/5
print(pred_blend)


[ 0.21756248  0.01050822  0.00486002 ...,  0.08707854  0.03200448
  0.00397106]


# Submit result

In [16]:

# prediction
# df_test_raw = pd.read_feather('/home/kai/data/talkingdata/downsampling/test_cleaned.ftr')
# print('loading file done!')
# df_test_raw = pd.read_csv('/home/kai/talkingdata/data/test.csv')
sub_dict = {}
df_sub = pd.DataFrame()
test_clickid = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv', usecols=['click_id'])
print('done')
df_sub['click_id'] = test_clickid['click_id']
df_sub['is_attributed'] = pred_blend
print('predicting file done!')
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/downsampling/4th_subblend1.csv.gz', compression='gzip', index=False)


done
predicting file done!


In [ ]:
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/downsampling/4th_sub.csv.gz', compression='gzip', index=False)

In [ ]:
a = df_sub.head(10)
# a = df_sub.sort_values(by=['click_id'])

# df_sub.to_csv('/home/kai/data/kaggle/talkingdata/downsampling/4th_sub.csv.gz', compression='gzip', index=False)

In [ ]:
import pandas as pd

xxx = pd.read_csv('/home/kai/data/kaggle/talkingdata/downsampling/4th_sub.csv.gz')
print('done')
test_clickid = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv', usecols=['click_id'])
print('done')
xxx['click_id'] = test_clickid['click_id']

In [3]:
xxx.to_csv('/home/kai/data/kaggle/talkingdata/downsampling/4th_sub.csv.gz', compression='gzip', index=False)